## Imports

In [1]:
!pip install --upgrade openai --quiet

     |████████████████████████████████| 44 kB 2.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 147 kB 9.0 MB/s 


In [2]:
!pip install openai --quiet

## API key

In [3]:
import openai
import os

os.environ["OPENAI_API_KEY"]=("YOURAPI")

# Data

Loading data

In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/jorgenhw/random_data/main/sangtekster.csv", sep = ";")
df = df.drop("ID", axis = 1) # dropping original ID column (was incorrect)
df['text'] = df['text'].str[4:] # deleting 4 first characters to remove first number from each song

Preprocessing

In [2]:
# Function extracts the first characters of a song until the first number occurs (indicating the beginning og verse 2)
def extract_until_number(s):
    result = ""
    for c in s:
        if c.isnumeric():
            break
        result += c
    return result


# Apply the above function to the text column in the df
df["prompt"] = df["text"].apply(extract_until_number)

# Delete the extracted characters from the "text" column
df["text"] = df.apply(lambda row: row["text"].replace(row["prompt"], ""), axis=1)

# rename
df = df.rename(columns={"text": "completion"})

# Reordering columns
df = df.reindex(columns=['prompt', 'completion'])

**Following songs are dropped from the dataset since we use the first verses from these as prompts**
* 311 (yndigt at følges ad), 
* 170 (hvidt herude), 
* 136 (der er noget i luften), 
* 132 (vær velkommen),  
* 301 (vem kan segla)
* 220 (danmark nu blunder)
* 140 (en rose så jeg skyde)
* 62 (jens vejmand)
* 305 (det var en lørdag aften)
* 321 (dansevise)
* 37 (amazing grace)

NB: Number refers to index number, not actual song number

In [3]:
songs = df.iloc[[311,170,136,132,301,220,140,62,305,321,37]]
prompts = songs["prompt"].tolist()

df = df.drop([311, 170,136,132,301,220,140,62,305,321,37])

In [5]:
df.shape

(402, 2)

In [89]:
# Writing file do drive
df.to_csv("processed_data.csv", index=False)
path = "/content/processed_data.csv"

Reformatting to jsonl


In [90]:
# From OpenAI's own docu
!openai tools fine_tunes.prepare_data -f $path

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 402 prompt-completion pairs
- `completion` column/key should not contain empty strings. These are rows: [186]
- There are 9 duplicated prompt-completion sets. These are rows: [95, 112, 149, 150, 168, 248, 276, 383, 397]
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- All completions end with suffix `\n`
` more than once. We suggest that you review your completions and add a unique ending
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://beta.

## Create a fine-tuned model

The following assumes you've already prepared training data following the above instructions.

Start your fine-tuning job using the OpenAI CLI:

In [95]:
train_file = "/content/processed_data_prepared.jsonl"

### Hyperparameters

In [96]:
model = 'davinci'  # can be ada, babbage, curie or davinci
n_epochs = 4
batch_size = 4
learning_rate_multiplier = 0.1
prompt_loss_weight = 0.1

### Training

In [97]:
!openai api fine_tunes.create \
    -t $train_file \
    -m $model \
    --n_epochs $n_epochs \
    --batch_size $batch_size \
    --learning_rate_multiplier $learning_rate_multiplier \
    --prompt_loss_weight $prompt_loss_weight

Upload progress: 100% 403k/403k [00:00<00:00, 785Mit/s]
Uploaded file from /content/processed_data_prepared.jsonl: file-qm5kNN8ibOkozvLdnmx7IAYW
Created fine-tune: ft-qEzBbVEURqzI2svgwhbSd0UC
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-12-28 10:03:12] Created fine-tune: ft-qEzBbVEURqzI2svgwhbSd0UC
[2022-12-28 10:04:31] Fine-tune costs $19.52
[2022-12-28 10:04:31] Fine-tune enqueued. Queue number: 0
[2022-12-28 10:04:33] Fine-tune started
[2022-12-28 10:09:20] Completed epoch 1/4

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-qEzBbVEURqzI2svgwhbSd0UC



In [100]:
!openai api fine_tunes.follow -i ft-qEzBbVEURqzI2svgwhbSd0UC

[2022-12-28 10:03:12] Created fine-tune: ft-qEzBbVEURqzI2svgwhbSd0UC
[2022-12-28 10:04:31] Fine-tune costs $19.52
[2022-12-28 10:04:31] Fine-tune enqueued. Queue number: 0
[2022-12-28 10:04:33] Fine-tune started
[2022-12-28 10:09:20] Completed epoch 1/4
[2022-12-28 10:12:06] Completed epoch 2/4
[2022-12-28 10:14:51] Completed epoch 3/4
[2022-12-28 10:17:34] Completed epoch 4/4
[2022-12-28 10:18:18] Uploaded model: davinci:ft-aarhus-university-2022-12-28-10-18-17
[2022-12-28 10:18:19] Uploaded result file: file-fjmIj2AlKlQik2UuUN9L7bZ2
[2022-12-28 10:18:19] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m davinci:ft-aarhus-university-2022-12-28-10-18-17 -p <YOUR_PROMPT>


## Inferencing

In [161]:
import os
import openai

def open_file(filepath):
  with open(filepath, 'r', encoding='utf-8') as infile:
    return infile.read()
# API key
openai.api_key = open_file("/content/openaiAPI.txt")

def gpt3(prompt):
  response = openai.Completion.create(
  model="davinci:ft-aarhus-university-2022-12-28-10-18-17",
  prompt=prompt,
  #stop = "\n",
  temperature=0.7,
  max_tokens=180,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
  )
  text = response['choices'][0]['text'].strip()
  return text

In [139]:
prompts

['Det er så yndigt at følges ad for to, som gerne vil sammen være, da er med glæden man dobbelt glad og halvt om sorgen så tung at bære; ja, det er gammen //: at rejse sammen, :// når fjederhammen //: er kærlighed. ://\n',
 'Det er hvidt herude,kyndelmisse slår sin knudeovermåde hvas og hård,hvidt forneden, hvidt foroven,pudret tykt står træ i skovensom udi min abildgård.\n',
 'Der er noget i luften, jeg ved ikke hvad,som forår, skønt skoven har mistet hvert blad.Der er noget i luften, som rosernes duften,som fuglenes fryd,skønt rosen er falmet, og fuglen er draget mod syd.\n',
 'Vær velkommen, Herrens år,og velkommen herhid!Julenat, da vor Herre blev fød,da tændte sig lyset i mørkets skød.Velkommen, nytår, og velkommen her!\n',
 'Vem kan segla förutan vind?Vem kan ro utan åror?Vem kan skiljas från vännen sinutan att fälla tårar?\n',
 'Danmark, nu blunder den lyse natbag ved din seng, når du sover.Gøgen kukker i skov og krat,Vesterhavet og Kattegatsynger, imens det dugger,sagte som san

In [178]:
gpt3(prompts[10])

"2. 'Twas grace that taught my heart to fear,And grace my fears relieved;How precious did that grace appearThe hour I first believed.\n\n3. Through many dangers, toils and snaresI have already come;'Tis grace hath brought me safe thus far,And grace will lead me home.\n\n4. The Lord has promised good to me,His word my hope secures;He will my shield and portion beAs long as life endures.\n\n5. Yea, when this flesh and heart shall fail,And mortal life shall cease,I shall possess, within the veil,A life of joy and peace.\n\n6. The earth shall soon dissolve like snow,The sun forbear to shine;But God, who called me here below,Will be forever mine.\n\n7. When we've been there"

In [176]:
prompts

['Det er så yndigt at følges ad for to, som gerne vil sammen være, da er med glæden man dobbelt glad og halvt om sorgen så tung at bære; ja, det er gammen //: at rejse sammen, :// når fjederhammen //: er kærlighed. ://\n',
 'Det er hvidt herude,kyndelmisse slår sin knudeovermåde hvas og hård,hvidt forneden, hvidt foroven,pudret tykt står træ i skovensom udi min abildgård.\n',
 'Der er noget i luften, jeg ved ikke hvad,som forår, skønt skoven har mistet hvert blad.Der er noget i luften, som rosernes duften,som fuglenes fryd,skønt rosen er falmet, og fuglen er draget mod syd.\n',
 'Vær velkommen, Herrens år,og velkommen herhid!Julenat, da vor Herre blev fød,da tændte sig lyset i mørkets skød.Velkommen, nytår, og velkommen her!\n',
 'Vem kan segla förutan vind?Vem kan ro utan åror?Vem kan skiljas från vännen sinutan att fälla tårar?\n',
 'Danmark, nu blunder den lyse natbag ved din seng, når du sover.Gøgen kukker i skov og krat,Vesterhavet og Kattegatsynger, imens det dugger,sagte som san

In [134]:
for i in prompts:
  print(gpt3(i))

#gpt3(prompts[0])

3 Det er så yndigt at følges ad for to, da lyses det lille ud af mørket i vor hjerte bag lås og bom, hvor følelser //: ligger gemt. :// Og det er så yndigt at følges ad for to, at milde ens hede sammen uden at brænde af.
4 Men se, det er syndigt, at følges ad for to, hvis ikke kærligheden blomstrer i os og er stedet, hvor følelserne //: ligger gemt. :// Ja, se, det er syndigt, //: at rejse sammen, :// når fjederhammen //: er kærlighed. ://
5 Ja, se, det er syndigt, at følges ad for to, når ejendomme skiller os fra hinanden, og vort hjerte bag lås og bom er fangernes fæ
###

 2. Hvidt er kornet og hvidt er byg,hvidt er svinene på gården,hvidt er høet i agerneog hvidt er jeg, som haver vågenhvisker til dig, min ven:3. Hvad er nu dette for et land,hvor det ikke er et blad til skue,kun en klæbende sne,kun en svedig kulde,kun et kornet korn i dit håndtegned ud på hjemvejen!4. Hvad er nu dette for et land,hvor der ikke er et blad at se,kun en snærende kulde,kun en klæbende sne,kun en svedig 